In [1]:
from random import choices, randint
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import random

# Configuration
highways = [
    {
        'id': 0,
        'name': "Frankfurter Allee",
        'electric_vehicle_rate': 0.2,
        'traffic_patterns': [
            {"condition": lambda h: h >= 22 or h < 5, "base_rate": 20, "speed": 40},
            {"condition": lambda h: 7 <= h < 10, "base_rate": 80, "speed": 32},  # Early rush
            {"condition": lambda h: 17 <= h < 19, "base_rate": 70, "speed": 35},
            {"condition": lambda h: True, "base_rate": 40, "speed": 38}
        ]
    },
    {
        'id': 1,
        'name': "Kaiserdamm",
        'electric_vehicle_rate': 0.18,
        'traffic_patterns': [
            {"condition": lambda h: h >= 23 or h < 6, "base_rate": 15, "speed": 42},
            {"condition": lambda h: 8 <= h < 10, "base_rate": 60, "speed": 35},
            {"condition": lambda h: 16 <= h < 18, "base_rate": 55, "speed": 38},
            {"condition": lambda h: True, "base_rate": 30, "speed": 39}
        ]
    },
    {
        'id': 2,
        'name': "Tempelhofer Damm",
        'electric_vehicle_rate': 0.12,
        'traffic_patterns': [
            {"condition": lambda h: h >= 21 or h < 7, "base_rate": 10, "speed": 45},
            {"condition": lambda h: 9 <= h < 11, "base_rate": 40, "speed": 37},  # Later morning peak
            {"condition": lambda h: 15 <= h < 17, "base_rate": 35, "speed": 40},
            {"condition": lambda h: True, "base_rate": 20, "speed": 43}
        ]
    },
    {
        'id': 3,
        'name': "Leipziger Straße",
        'electric_vehicle_rate': 0.21,
        'traffic_patterns': [
            {"condition": lambda h: h >= 20 or h < 4, "base_rate": 25, "speed": 42},
            {"condition": lambda h: 5 <= h < 8, "base_rate": 70, "speed": 40},  # Very early morning commute
            {"condition": lambda h: 14 <= h < 16, "base_rate": 60, "speed": 38},  # Afternoon shift change
            {"condition": lambda h: True, "base_rate": 45, "speed": 41}
        ]
    }
]


LETTERS = 'ABCDEFGHIJKLMNOPQRSTUVYZ'

def gen_number_plate():
    letters_city = ''.join(choices(LETTERS, k=randint(1,2)))
    letters_mid  = ''.join(choices(LETTERS, k=randint(1,2)))
    numbers = randint(1, 9999)
    return f'{letters_city}-{letters_mid} {numbers:03d}'

def generate_traffic_data():
    start_time = datetime(2025, 1, 31, 0, 0, 0)
    total_minutes = 24 * 60
    data = []

    for minute_offset in range(total_minutes):
        current_time = start_time + timedelta(minutes=minute_offset)
        hour = current_time.hour

        for highway in highways:
            # Get base rate for current hour
            base_rate = next(
                p["base_rate"] for p in highway['traffic_patterns']
                if p["condition"](hour)
            )
            base_speed = next(
                p["speed"] for p in highway['traffic_patterns']
                if p["condition"](hour)
            )
            
            
            # Add random variance (+/- 20%)
            varied_rate = base_rate * np.random.uniform(0.8, 1.2)
            cars_this_minute = np.random.poisson(varied_rate)

            for _ in range(cars_this_minute):
                seconds_offset = np.random.uniform(0, 60)
                
                plate = gen_number_plate()

                if random.random() < highway['electric_vehicle_rate']:
                    electric_vehicle = True
                    plate += 'E'
                else:
                    electric_vehicle = False 
                    
                car_time = current_time + timedelta(seconds=seconds_offset)
                car_speed = round(base_speed * np.random.uniform (0.8, 1.2),3)
                data.append({
                    'timestamp': car_time,
                    'highway_id': highway['id'],
                    'plate': plate,
                    'speed': car_speed,
                    'ev': electric_vehicle
                })

    df = pd.DataFrame(data)
    df = df.sort_values('timestamp').reset_index(drop=True)
    return df

# Generate and save data
df = generate_traffic_data()
df.to_csv("multi_road_traffic.csv", index=False)
print(f"Generated {len(df)} car passages across 4 highways")
print("Sample data:")
print(df.head())

Generated 192586 car passages across 4 highways
Sample data:
                   timestamp  highway_id      plate   speed     ev
0 2025-01-31 00:00:00.627353           1   Z-A 4226  37.416  False
1 2025-01-31 00:00:00.746412           1  EP-P 8099  34.365  False
2 2025-01-31 00:00:02.359477           3   B-N 9804  35.364  False
3 2025-01-31 00:00:03.757061           1  C-PA 3669  47.668  False
4 2025-01-31 00:00:04.557995           0   H-A 6079  36.379  False
